In [1]:

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
import os
import numpy as np
import torch
# # import onnx
import onnxruntime as ort
import torch.nn as nn

# from typing import Sequence, NamedTuple

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cuda:0'

In [3]:
# cfg_file = "./onnx_config.yaml"
cfg_file = "./pillar/config.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_pillar_sfaw')==False:
    os.mkdir('./deploy_pillar_sfaw')
log_file = './deploy_pillar_sfaw/log_trt.log'
logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

2024-03-09 05:15:36,408   INFO  Loading Custom dataset.
2024-03-09 05:15:36,410   INFO  Total samples for CUSTOM dataset: 0


In [4]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
# cfg

In [5]:
ckpt = "./pillar/model.pth"
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()

2024-03-09 05:15:41,810   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-03-09 05:15:41,936   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-03-09 05:15:41,965   INFO  ==> Done (loaded 391/391)


CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [6]:
# batch_dict = torch.load("./batch_dict.pth", map_location="cuda")
# inputs = batch_dict
# inputs

In [7]:
batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth")
inputs = {}
inputs['batch_dict']={}
# inputs = batch_dict
# inputs.to(device)
inputs['batch_dict']['points'] = batch_dict['points']
inputs['batch_dict']['lidar_aug_matrix'] = batch_dict['lidar_aug_matrix']
inputs['batch_dict']['use_lead_xyz'] = batch_dict['use_lead_xyz']
inputs['batch_dict']['batch_size'] = torch.tensor(batch_dict['batch_size']).to(device)

In [8]:
# inputs['batch_dict'] =inputs
inputs

{'batch_dict': {'points': tensor([[ 0.0000,  0.8040,  0.7984, -1.4852,  0.0000],
          [ 0.0000,  0.8203,  0.7921, -1.4947,  0.0000],
          [ 0.0000,  0.8500,  0.8309, -1.4720,  0.0000],
          ...,
          [ 0.0000, -0.6915,  0.6910,  1.2054,  0.0000],
          [ 0.0000, -0.6724,  0.6628,  1.2318,  0.0000],
          [ 0.0000, -0.6677,  0.6675,  1.2318,  0.0000]], device='cuda:0'),
  'lidar_aug_matrix': tensor([[[1., 0., 0., 0.],
           [0., 1., 0., 0.],
           [0., 0., 1., 0.],
           [0., 0., 0., 1.]]], device='cuda:0'),
  'use_lead_xyz': tensor([1.], device='cuda:0'),
  'batch_size': tensor(1, device='cuda:0')}}

In [9]:

jit_mode = "trace"
input_names = [
    'batch_dict'
]
output_names = ["output"]

# dynamic_axes = {
#     "src": {
#         0: "voxel_number",
#     },
#     "set_voxel_inds_tensor_shift_0": {
#         1: "set_number_shift_0",
#     },
#     "set_voxel_inds_tensor_shift_1": {
#         1: "set_number_shift_1",
#     },
#     "set_voxel_masks_tensor_shift_0": {
#         1: "set_number_shift_0",
#     },
#     "set_voxel_masks_tensor_shift_1": {
#         1: "set_number_shift_1",
#     },
#     "pos_embed_tensor": {
#         2: "voxel_number",
#     },
#     "output": {
#         0: "voxel_number",
#     }
# }

In [ ]:
base_name = "./deploy_pillar_sfaw/dsvt_all"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

# allptransblocktrt = AllDSVTBlocksTRT(dsvtblocks_list, layer_norms_list).eval().cuda()

torch.onnx.export(
    model,
    inputs,
    onnx_path, input_names=input_names,
    output_names=output_names, 
    # dynamic_axes=dynamic_axes,
    opset_version=14,
)
# https://github.com/Haiyang-W/DSVT/issues/60

/opt/dec/pcdet/models/model_utils/dsvt_utils.py:135: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  set_num = setnum_per_win.sum().item() # set_num = 7
/opt/dec/pcdet/models/backbones_3d/dsvt.py:601: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert coors_in_win.size(1) == 3
/opt/dec/pcdet/models/backbones_2d/map_to_bev/pointpillar_scatter.py:53: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This mea

In [ ]:
# test onnx
ort_session = ort.InferenceSession(onnx_path)
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(pillar_features),
              ort_session.get_inputs()[1].name: to_numpy(set_voxel_inds_list[0][0]),
              ort_session.get_inputs()[2].name: to_numpy(set_voxel_inds_list[0][1]),
              ort_session.get_inputs()[3].name: to_numpy(set_voxel_masks_list[0][0]),
              ort_session.get_inputs()[4].name: to_numpy(set_voxel_masks_list[0][1]),
              ort_session.get_inputs()[5].name: to_numpy(torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0)),}
ort_outs = ort_session.run(None, ort_inputs)
ort_outs[0].shape
